## Method 1

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [2]:
ds = tfds.load('squad', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)

In [10]:
for i in ds:
    pass

## Method 2 (Used)

In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import pandas as pd

In [2]:
f = open('train-v2.0.json')
json_file = json.load(f)

In [ ]:
title = []
context = []
question = []
answer = []
for i in json_file['data']:
    title.append(i['title'])
    tempq = []
    tempa = []
    tempc = ''
    for j in i['paragraphs']:
        tempc += j['context'] + ' '
        for k in j['qas']:
            if len(k['answers']) == 0:
                continue
            tempq.append(k['question'])
            tempa.append(k['answers'][0])
    context.append(tempc)
    question.append(tempq)
    answer.append(tempa)

In [10]:
df = pd.DataFrame({'title':title, 'context':context, 'question':question, 'answer':answer})

In [15]:
df.to_pickle('data.df')

In [16]:
df = pd.read_pickle('data.df')

In [17]:
for i in range(len(df)):
    if len(df['question'][i])!=len(df['answer'][i]):
        print('lol')

## Prepare question generation dataset from SQuaD

In [1]:
from nltk.tokenize import sent_tokenize
import json
import pandas as pd

In [2]:
f = open('dev-v2.0.json')
json_file = json.load(f)

In [3]:
qg_dataset = []
num_qa = 0
num_exact = 0 

for i in json_file['data']:
    title = i['title']
    context = []
    question = []
    answer = []
    for j in i['paragraphs']:
        for k in j['qas']:
            if len(k['answers']) == 0:
                continue
            
            num_qa+=1
            qg_row = {}

            qg_row['question'] = k['question']
            sentences = sent_tokenize(j['context'])
            found = False
            for ans in k['answers']:
                for s in range(len(sentences)):
                    if ans['text'] in sentences[s] and not found:
                        num_exact+=1
                        sent = sentences[s]
                        found = True
                        break
                    
            qg_row['sentence'] = sent
            qg_row['answer'] = ans['text']
            qg_row['context'] = j['context']
            qg_row['exact'] = found
            qg_dataset.append(qg_row)
            
print(f"Exact span found for {round(num_exact/num_qa, 3)} of data")

Exact span found for 0.999 of data


In [4]:
qg_df = pd.DataFrame(qg_dataset)

In [5]:
qg_df.iloc[0].to_dict()

{'question': 'In what country is Normandy located?',
 'sentence': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France.',
 'answer': 'France',
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeedin

In [6]:
# failed to identify sentence containing answer
qg_df[qg_df['exact']==False]

,question,sentence,answer,context,exact
899,Who was one prominent Huguenot-descended arms ...,The Huguenots adapted quickly and often marrie...,E.I. du Pont,Most of the Huguenot congregations (or individ...,False
2334,Who allegedly coined the name Black Death?,Gasquet (1908) claimed that the Latin name atr...,J.I. Pontanus,Gasquet (1908) claimed that the Latin name atr...,False
3176,What famous civil rights leader called Harvard...,Du Bois; philosopher Henry David Thoreau; auth...,W. E. B. Du Bois,Other: Civil rights leader W. E. B. Du Bois; p...,False
3734,Who calculated the speed of light?,Other prominent alumni include anthropologists...,A. A. Michelson,Notable faculty in physics have included the s...,False
4403,"According to Wilson's theorem, what factorial ...",Another consequence of Fermat's little theorem...,(p − 1)! + 1,Giuga's conjecture says that this equation is ...,False


In [7]:
qg_df[qg_df['exact']==True].to_csv('qg_dev.csv', encoding='utf-8')